# BTC GARCH & Realized Variance Visualization

This notebook loads a JSONL file containing BTC 5-minute price data and GARCH model outputs, then:

- Computes rolling realized variance over 1h, 4h, and 1d windows (sum of squared log returns).
- Detects available GARCH-related variance/sigma columns automatically.
- Plots BTC price on the primary y-axis and realized variances plus GARCH estimates on the secondary y-axis using Plotly.

Adjust the `DATA_FILE` path below if needed.

In [50]:
# Load BTC 5‑min data + GARCH outputs and plot price vs realized & GARCH variances
import pandas as pd, numpy as np
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Path to data file (adjust if necessary)
DATA_FILE = Path("/Users/vobornij/projects/polybot/build/estimator/btc_5min_with_var.jsonl")

if not DATA_FILE.exists():
    raise FileNotFoundError(f"Data file not found: {DATA_FILE}")

# Load JSONL (expects at least: timestamp, price (or close), plus GARCH output columns)
df = pd.read_json(DATA_FILE, lines=True)

# Flexible price column detection
price_col_candidates = [c for c in ['price','close','btc_price','BTC'] if c in df.columns]
if not price_col_candidates:
    raise ValueError("Could not find a price column among expected candidates: price/close/btc_price/BTC")
PRICE_COL = price_col_candidates[0]

# Normalize timestamp column name
if 'timestamp' not in df.columns:
    # Try common alternatives
    ts_alts = [c for c in df.columns if c.lower() in {'time','datetime','date','ts'}]
    if not ts_alts:
        raise ValueError("Expected 'timestamp' column (or time/datetime/date/ts) not found.")
    df.rename(columns={ts_alts[0]: 'timestamp'}, inplace=True)

df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
df = df.sort_values('timestamp').reset_index(drop=True)

# Compute log returns on 5-min bars (if not already present)
if 'log_return' not in df.columns:
    df['prev_price'] = df[PRICE_COL].shift(1)
    df['log_return'] = np.log(df[PRICE_COL] / df['prev_price'])

# Window sizes in 5‑min bars (assuming constant sampling interval)
WIN_1H = 12          # 12 * 5m = 60m
WIN_4H = 12 * 4      # 48
WIN_1D = 12 * 24     # 288

# Realized variance (sum of squared log returns over window, then average per bar by dividing by count)
ret2 = df['log_return']**2

df['sigma'] = np.sqrt(df['sigma2'])

# 95th percentile cap (global) for squared returns to create a robust realized variance variant
cap_threshold = ret2.quantile(0.95)
# Basic percentile diagnostics
if 'ret2_percentile_stats_printed' not in df.attrs:
    print("Cap threshold (95th percentile of squared returns):", cap_threshold)
    for p in [0.5, 0.90, 0.99]:
        print(f"Percentile {p} of squared returns:", ret2.quantile(p))
    df.attrs['ret2_percentile_stats_printed'] = True

ret2_capped = ret2.clip(upper=cap_threshold)

# Standard realized variance (average per 5m bar over horizon)
df['rv_1h'] = ret2.rolling(WIN_1H, min_periods=WIN_1H).sum()/WIN_1H
df['rv_4h'] = ret2.rolling(WIN_4H, min_periods=WIN_4H).sum()/WIN_4H
df['rv_1d'] = ret2.rolling(WIN_1D, min_periods=WIN_1D).sum()/WIN_1D

# Capped 1h realized variance (average per 5m bar)
df['rv_1h_capped'] = ret2_capped.rolling(WIN_1H, min_periods=WIN_1H).sum()/WIN_1H
df['rv_4h_capped'] = ret2_capped.rolling(WIN_4H, min_periods=WIN_4H).sum()/WIN_4H

# Key variance columns explicitly handled
key_variance_cols = [
    ('sigma2', 'Model sigma2', '#d62728', 'solid', 1.2),
    ('sigma', 'Model sigma', '#d62728', 'solid', 1.2),
    ('backbone_sigma2', 'Backbone sigma2', '#9467bd', 'dash', 1),
    ('template_sigma2', 'Template sigma2', '#17becf', 'dot', 1.5)
]

# Automatically detect OTHER GARCH variance/sigma columns
keywords = ('garch','sigma','var','vol')
garch_cols = []
for c in df.columns:
    lc = c.lower()
    if any(k in lc for k in keywords):
        if c not in {'rv_1h','rv_4h','rv_1d','rv_1h_capped','rv_4h_capped', 'sigma2','backbone_sigma2','template_sigma2', 'seasonal_sigma_factor'} and pd.api.types.is_numeric_dtype(df[c]):
            garch_cols.append(c)

# Build figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Primary axis: BTC price
fig.add_trace(go.Scatter(
    x=df['timestamp'], y=df[PRICE_COL],
    name='BTC Price', mode='lines',
    line=dict(color='black')), secondary_y=False)

# Secondary axis: realized variances
rv_traces = [
    ('RV 1h', 'rv_1h', '#1f77b4'),
    ('RV 1h capped', 'rv_1h_capped', '#1f77b4'),  # same base color, will adjust style
    ('RV 4h', 'rv_4h', '#ff7f0e'),
    ('RV 4h capped', 'rv_4h_capped', '#ff7f0e'),  # same base color, will adjust style
    ('RV 1d', 'rv_1d', '#2ca02c'),
]
for label, col, color in rv_traces:
    style = dict(width=1, color=color)
    if 'capped' in label.lower():
        style['dash'] = 'dot'
    fig.add_trace(go.Scatter(
        x=df['timestamp'], y=df[col],
        name=label, mode='lines',
        line=style,
        opacity=0.85 if 'capped' not in label.lower() else 0.9), secondary_y=True)

# Secondary axis: key variance series (highlighted)
for col, nice_name, color, dash, width in key_variance_cols:
    if col in df.columns and pd.api.types.is_numeric_dtype(df[col]):
        fig.add_trace(go.Scatter(
            x=df['timestamp'], y=df[col], name=nice_name,
            mode='lines',
            line=dict(color=color, dash=dash, width=width*2),
            opacity=0.95), secondary_y=True)

# Secondary axis: OTHER GARCH outputs (lighter dotted)
palette = ['#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#1f77b4', '#ff9896']
for i, col in enumerate(sorted(garch_cols)):
    fig.add_trace(go.Scatter(
        x=df['timestamp'], y=df[col],
        name=col, mode='lines',
        line=dict(width=1, dash='dash', color=palette[i % len(palette)]),
        opacity=0.6), secondary_y=True)

fig.update_yaxes(title_text="BTC Price (USD)", secondary_y=False)
fig.update_yaxes(title_text="Variance / GARCH Outputs (per 5m avg)", secondary_y=True)

fig.update_layout(
    title="BTC Price vs Realized Variance (1h / 4h / 1d, capped 1h) and Model Variance Components",
    hovermode='x unified',
    legend=dict(orientation='h', y=1.02, x=0),
    height=750,
    margin=dict(l=60, r=40, t=60, b=40)
)

fig.show(renderer="browser")

# Display a quick list of detected variance-related columns
print("Detected additional variance columns (excluding key ones):")
print(garch_cols)
print(f"95th percentile squared return cap: {cap_threshold:.2e}")

# Ratios for quick diagnostics if template/backbone/model present
if all(col in df.columns for col in ['sigma2','backbone_sigma2','template_sigma2']):
    with np.errstate(divide='ignore', invalid='ignore'):
        ratio_model_template = (df['sigma2'] / df['template_sigma2']).replace([np.inf,-np.inf], np.nan)
        ratio_backbone_template = (df['backbone_sigma2'] / df['template_sigma2']).replace([np.inf,-np.inf], np.nan)
        print('Median sigma2 / template_sigma2:', ratio_model_template.median())
        print('Median backbone_sigma2 / template_sigma2:', ratio_backbone_template.median())

Cap threshold (95th percentile of squared returns): 8.24250682159075e-06
Percentile 0.5 of squared returns: 3.9543184733916117e-07
Percentile 0.9 of squared returns: 4.377826134165889e-06
Percentile 0.99 of squared returns: 2.6930827569600624e-05
Detected additional variance columns (excluding key ones):
['sigma']
95th percentile squared return cap: 8.24e-06
Detected additional variance columns (excluding key ones):
['sigma']
95th percentile squared return cap: 8.24e-06


## Compare model variance outputs to forward 1h realized variance

We now have additional fields in the data:

- `seasonal_sigma_factor`: seasonal multiplicative factor applied to volatility (sigma), not variance.
- `sigma2`: model conditional variance (after applying seasonality) per 5-minute bar.
- `backbone_sigma2`: underlying (deseasoned) variance component (should explain variance of `deseasoned_log_return`).
- `deseasoned_log_return`: return with intraday seasonal sigma removed (i.e., original log return divided by seasonal sigma factor if constructed that way).

Goals:
1. Compute forward 1h realized variance of raw log returns (`fwd_rv_1h`) and compare with `sigma2` (optionally aggregated to 1h).
2. Compute forward 1h realized variance of deseasoned log returns (`fwd_rv_deseasoned_1h`) and compare with `backbone_sigma2`.
3. Visualize all series.

Forward 1h realized variance at time t uses returns from (t, t+1h]; constructed via a shifted rolling sum of squared returns.

In [51]:
# Compute forward 1h realized variance (raw and deseasoned) and compare with model variance outputs
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Ensure needed columns exist
required_cols = {'log_return','sigma2','backbone_sigma2'}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# If deseasoned_log_return not present, attempt to reconstruct using seasonal_sigma_factor
if 'deseasoned_log_return' not in df.columns:
    if 'seasonal_sigma_factor' in df.columns:
        # Assuming original: log_return = deseasoned_log_return * seasonal_sigma_factor
        # Then deseasoned = log_return / seasonal_sigma_factor
        df['deseasoned_log_return'] = df['log_return'] / df['seasonal_sigma_factor']
    else:
        df['deseasoned_log_return'] = np.nan

BAR_PER_HOUR = 12

# Forward realized variance: sum_{t+1 to t+12} r^2. We can use rolling sum on shifted series.
# Shift - (negative) moves future returns backward so they align with current timestamp.
ret2 = df['log_return']**2
fwd_rv_raw = ret2.shift(-1).rolling(BAR_PER_HOUR, min_periods=BAR_PER_HOUR).sum()
# Deseasoned
ret2_des = df['deseasoned_log_return']**2
fwd_rv_des = ret2_des.shift(-1).rolling(BAR_PER_HOUR, min_periods=BAR_PER_HOUR).sum()

# Store
df['fwd_rv_1h'] = fwd_rv_raw
df['fwd_rv_deseasoned_1h'] = fwd_rv_des

# Optionally scale model variance to 1h horizon for comparison (sum of next 12 conditional variances)
# We'll compute forward 1h model variance by summing sigma2 forward (and backbone separately)
model_var_fwd = df['sigma2'].shift(-1).rolling(BAR_PER_HOUR, min_periods=BAR_PER_HOUR).sum()
backbone_var_fwd = df['backbone_sigma2'].shift(-1).rolling(BAR_PER_HOUR, min_periods=BAR_PER_HOUR).sum()

df['fwd_model_var_1h'] = model_var_fwd
df['fwd_backbone_var_1h'] = backbone_var_fwd

# Template variance forward 1h if available
if 'template_sigma2' in df.columns and pd.api.types.is_numeric_dtype(df['template_sigma2']):
    df['fwd_template_var_1h'] = df['template_sigma2'].shift(-1).rolling(BAR_PER_HOUR, min_periods=BAR_PER_HOUR).sum()
else:
    df['fwd_template_var_1h'] = np.nan

# Build comparison figure
fig2 = make_subplots(specs=[[{"secondary_y": True}]])

# Primary y: BTC price
fig2.add_trace(go.Scatter(
    x=df['timestamp'], y=df[PRICE_COL], name='BTC Price',
    line=dict(color='black'), opacity=0.7), secondary_y=False)

# Secondary y: variance series (raw)
fig2.add_trace(go.Scatter(
    x=df['timestamp'], y=df['fwd_rv_1h'], name='Forward 1h RV (raw)',
    line=dict(color='#1f77b4')), secondary_y=True)
fig2.add_trace(go.Scatter(
    x=df['timestamp'], y=df['fwd_model_var_1h'], name='Forward 1h Model Var (sigma2 sum)',
    line=dict(color='#ff7f0e')), secondary_y=True)

# Template variance (if present)
if df['fwd_template_var_1h'].notna().any():
    fig2.add_trace(go.Scatter(
        x=df['timestamp'], y=df['fwd_template_var_1h'], name='Forward 1h Template Var (sum)',
        line=dict(color='#17becf', dash='dot')), secondary_y=True)

# Secondary y: deseasoned comparison (dotted)
fig2.add_trace(go.Scatter(
    x=df['timestamp'], y=df['fwd_rv_deseasoned_1h'], name='Forward 1h RV (deseasoned)',
    line=dict(color='#2ca02c', dash='dot')), secondary_y=True)
fig2.add_trace(go.Scatter(
    x=df['timestamp'], y=df['fwd_backbone_var_1h'], name='Forward 1h Backbone Var (sum)',
    line=dict(color='#d62728', dash='dash')), secondary_y=True)

fig2.update_yaxes(title_text='BTC Price (USD)', secondary_y=False)
fig2.update_yaxes(title_text='Variance (1h forward)', secondary_y=True)
fig2.update_layout(title='Forward 1h Realized Variance vs Model / Template / Backbone Variance Components',
                   hovermode='x unified', legend=dict(orientation='h', y=1.03, x=0), height=700)
fig2.show(renderer='browser')

# Basic evaluation metrics: correlations
valid_mask_raw = df['fwd_rv_1h'].notna() & df['fwd_model_var_1h'].notna()
valid_mask_template = df['fwd_rv_1h'].notna() & df['fwd_template_var_1h'].notna()
valid_mask_des = df['fwd_rv_deseasoned_1h'].notna() & df['fwd_backbone_var_1h'].notna()

if valid_mask_raw.sum() > 10:
    corr_raw = np.corrcoef(df.loc[valid_mask_raw, 'fwd_rv_1h'], df.loc[valid_mask_raw, 'fwd_model_var_1h'])[0,1]
    print(f"Correlation (raw): forward RV vs model var = {corr_raw:.3f}")
if valid_mask_template.sum() > 10:
    corr_template = np.corrcoef(df.loc[valid_mask_template, 'fwd_rv_1h'], df.loc[valid_mask_template, 'fwd_template_var_1h'])[0,1]
    print(f"Correlation (raw): forward RV vs template var = {corr_template:.3f}")
if valid_mask_des.sum() > 10:
    corr_des = np.corrcoef(df.loc[valid_mask_des, 'fwd_rv_deseasoned_1h'], df.loc[valid_mask_des, 'fwd_backbone_var_1h'])[0,1]
    print(f"Correlation (deseasoned): forward RV vs backbone var = {corr_des:.3f}")

# Relative error metrics (MAE) for raw realized variance comparisons
if valid_mask_raw.sum() > 10:
    mae_model = np.mean(np.abs(df.loc[valid_mask_raw, 'fwd_rv_1h'] - df.loc[valid_mask_raw, 'fwd_model_var_1h']))
    print(f"MAE raw RV vs model var: {mae_model:.2e}")
if valid_mask_template.sum() > 10:
    mae_template = np.mean(np.abs(df.loc[valid_mask_template, 'fwd_rv_1h'] - df.loc[valid_mask_template, 'fwd_template_var_1h']))
    print(f"MAE raw RV vs template var: {mae_template:.2e}")

Correlation (raw): forward RV vs model var = 0.669
Correlation (deseasoned): forward RV vs backbone var = 0.633
MAE raw RV vs model var: 1.39e-05


In [52]:
# Weekly seasonal sigma template: EXPECTS new structure (sigmaEwmShrunkNormalized, sigmaEwmFactor) without defensive checks
import json, pandas as pd, numpy as np
from datetime import datetime, timezone, timedelta
import plotly.graph_objects as go
from pathlib import Path

TEMPLATE_FILE = Path("/Users/vobornij/projects/polybot/build/estimator/btc_sigma_template_5min.json")
with open(TEMPLATE_FILE, "r") as f:
    tpl = json.load(f)

avg_mu_5m = tpl["averageMu5min"]

rows = []
for e in tpl["template"]:  # assume list exists
    k = e["key"]
    day = k["day"]
    hour = k["hour"]
    idx5 = k["interval5minIndex"]
    weekly_interval = day*24*12 + hour*12 + idx5
    sigma = e["sigma"]
    sigma_norm = e["sigmaEwmShrunkNormalized"]
    sigma_factor = e["sigmaEwmFactor"]
    seasonal_sigma = sigma_norm * sigma_factor
    rows.append({
        "day": day,
        "hour": hour,
        "interval5": idx5,
        "weekly_interval": weekly_interval,
        "sigma": sigma,
        "sigmaEwmShrunkNormalized": sigma_norm,
        "sigmaEwmFactor": sigma_factor,
        "seasonalSigma": seasonal_sigma,
    })

df_tpl = pd.DataFrame(rows).sort_values("weekly_interval").reset_index(drop=True)
expected = 7*24*12
if len(df_tpl) != expected:
    raise ValueError(f"Template length {len(df_tpl)} != expected {expected}")

# Time alignment (last completed UTC week start)
now = datetime.now(timezone.utc)
start_current_week = (now - timedelta(days=now.weekday())).replace(hour=0, minute=0, second=0, microsecond=0)
week_start = start_current_week - timedelta(days=7)
df_tpl["week_time_utc"] = week_start + pd.to_timedelta(df_tpl["weekly_interval"]*5, unit="m")

# Require all needed columns
required_cols = ["sigma","sigmaEwmShrunkNormalized","sigmaEwmFactor","seasonalSigma"]
missing = [c for c in required_cols if c not in df_tpl.columns]
if missing:
    raise KeyError(f"Missing required columns in template: {missing}")

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_tpl['week_time_utc'], y=df_tpl['sigma'], name='Sigma (raw)', mode='lines', line=dict(color='#17becf', width=1.2)))
fig.add_trace(go.Scatter(x=df_tpl['week_time_utc'], y=df_tpl['seasonalSigma'], name='Seasonal σ (norm * factor)', mode='lines', line=dict(color='#ff7f0e', width=1.4)))
fig.add_trace(go.Scatter(x=df_tpl['week_time_utc'], y=df_tpl['sigmaEwmShrunkNormalized'], name='σ Norm (shape)', mode='lines', line=dict(color='#2ca02c', dash='dot', width=1)))
fig.add_trace(go.Scatter(x=df_tpl['week_time_utc'], y=df_tpl['sigmaEwmFactor'], name='σ Factor', mode='lines', line=dict(color='#9467bd', dash='dash', width=1)))

fig.update_layout(
    title=f"Weekly Seasonal Sigma Pattern (last completed UTC week start {week_start.date()})",
    xaxis_title="UTC Time (last completed week)",
    yaxis_title="Sigma / Components",
    hovermode='x unified',
    legend=dict(orientation='h', y=1.02, x=0),
    height=420,
    margin=dict(l=55, r=30, t=55, b=40)
)
for d in range(1,7):
    fig.add_vline(x=week_start + timedelta(days=d), line_width=1, line_dash='dot', line_color='rgba(90,90,90,0.35)')
fig.show(renderer='browser')

print(f"averageMu5min={avg_mu_5m}")

averageMu5min=0.0


In [53]:
from pathlib import Path
import json, pandas as pd, plotly.graph_objects as go, numpy as np

# Realized variance file (5m)
rv_path = Path('/Users/vobornij/projects/polymarket/data/btc/rv/5m/btc_rv5m_20250519_000000_to_20250526_000000.jsonl')
rows = [json.loads(l) for l in rv_path.read_text().splitlines() if l.strip()]
rv_df = pd.DataFrame(rows)
rv_df['intervalStart'] = pd.to_datetime(rv_df['intervalStart'])
rv_df['intervalEnd'] = pd.to_datetime(rv_df['intervalEnd'])
rv_df['mid_ts'] = rv_df['intervalStart'] + (rv_df['intervalEnd'] - rv_df['intervalStart'])/2
rv_df = rv_df.sort_values('mid_ts').reset_index(drop=True)

# 1h rolling realized variance (sum of 12 consecutive 5m RV values), normalize per-5m, then convert to sigma
rv_df['rv_1h_sum'] = rv_df['realizedVariance'].rolling(12, min_periods=12).sum()
rv_df['rv_1h_per5m'] = rv_df['rv_1h_sum'] / 12.0

# Convert variances to standard deviations (per 5m)
rv_df['sigma_5m'] = np.sqrt(rv_df['realizedVariance'])
rv_df['sigma_1h_per5m'] = np.sqrt(rv_df['rv_1h_per5m'])

# Crop model variance to same time span and convert to sigma
mask = (df['timestamp'] >= rv_df['intervalStart'].min()) & (df['timestamp'] <= rv_df['intervalEnd'].max())
df_sigma = df.loc[mask, ['timestamp','sigma2']].dropna().copy()
df_sigma['sigma'] = np.sqrt(df_sigma['sigma2'])


rv_df['weekly_interval'] = (rv_df['mid_ts'].dt.dayofweek * 24 * 12
                            + rv_df['mid_ts'].dt.hour * 12
                            + (rv_df['mid_ts'].dt.minute // 5))
pattern = df_tpl.set_index('weekly_interval')['seasonalSigma']
seasonal_sigma = pattern.reindex(rv_df['weekly_interval']).values

fig = go.Figure()
fig.add_trace(go.Scatter(x=rv_df['mid_ts'], y=rv_df['sigma_5m'], name='Sigma 5m (realized)', mode='lines', line=dict(color='#1f77b4', width=1)))
fig.add_trace(go.Scatter(x=rv_df['mid_ts'], y=rv_df['sigma_1h_per5m'], name='Sigma 1h (per 5m avg)', mode='lines', line=dict(color='#2ca02c', dash='dash', width=2), opacity=0.9))
fig.add_trace(go.Scatter(x=df_sigma['timestamp'], y=df_sigma['sigma'], name='Sigma (model)', mode='lines', line=dict(color='#d62728', width=1)))
fig.add_trace(go.Scatter(x=rv_df['mid_ts'], y=seasonal_sigma, name='Seasonal Sigma (template)', mode='lines', line=dict(color='#9467bd', dash='dot', width=2), opacity=0.85))

fig.update_layout(title='BTC 5m Realized Sigma vs Model Sigma + Seasonal Pattern',
                  yaxis_title='Sigma (per 5m)', xaxis_title='Time', hovermode='x unified',
                  legend=dict(orientation='h', y=1.02, x=0))
fig.show(renderer='browser')